# Import Libraries and Load Data

In [1]:
# Modelling Libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

import random
# NLP Libraries
import nltk
nltk.download('punkt')
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
# Utility Libraries
import numpy as np
import json
import pickle

lemmatizer = WordNetLemmatizer()
# load data
with open("intents.json", "r") as fh:
    intents = json.load(fh)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hlao1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hlao1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Data Preprocessing

In [2]:
# tokenization
words, classes, documents = [], [], []
ignore_chars = "! ? , .".split(" ")
for i in intents["intents"]:
    for p in i["patterns"]:
        word = nltk.word_tokenize(p)
        words.extend(word)
        documents.append((word, i['tag']))
        if i['tag'] not in classes:
            classes.append(i['tag'])

    

In [3]:
documents[:3]

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting')]

In [4]:
# lemmatization
# tranformation of words into a lemma form such that they can be reduced
# to canonical words
# Ex. play, played, plays, playing --> are all repr as play
# Reason: reduce number of total words in the model's vocab
words = [lemmatizer.lemmatize(w.lower()) 
         for w in words if w not in ignore_chars]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
# cache into binary files
with open("words.pkl", "wb") as fh:
    pickle.dump(words, fh)
with open("classes.pkl", "wb") as fh:
    pickle.dump(classes, fh)

# Create Training and Testing Set

In [5]:
x = []
Y_empty = [0]*len(classes)
for d in documents:
    bag = []
    word_patterns = d[0]
    word_patterns = [lemmatizer.lemmatize(w.lower()) for
                    w in word_patterns]
    for _ in words:
        bag.append(1) if _ in word_patterns else bag.append(0)
        
    Y_row = list(Y_empty) 
    Y_row[classes.index(d[1])] = 1
    x.append([bag, Y_row])
random.shuffle(x)
x = np.array(x)

x_train, Y_train = list(x[:,0]), list(x[:,1])


<ipython-input-5-d3ab29f3debf>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x = np.array(x)


In [6]:
x_train

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [7]:
Y_train

[[0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0,

# Create the Model
* https://keras.io/guides/sequential_model/
* https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

In [8]:
# use a deep neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(Y_train[0]), activation='softmax'))
# compile the model. SGD + Nesterov accel gradient yields good results 
# for the particular model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd,
              metrics=['accuracy'])
print("Completed model compilation...")
# train model
hist = model.fit(np.array(x_train), 
                 np.array(Y_train),
                 epochs=200, 
                 batch_size=5,
                 verbose=1)
print("Completed model fitting...")
# cache model state into binary
model.save("chatbot_model.h5", hist)
print("Completed model caching...")




Completed model compilation...
Epoch 1/200
10/10 [==============================] - 1s 4ms/step - loss: 2.2254 - accuracy: 0.0780
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1325 - accuracy: 0.3140
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0353 - accuracy: 0.2757
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 1.9705 - accuracy: 0.4487
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 1.8803 - accuracy: 0.4497
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7605 - accuracy: 0.5289
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7813 - accuracy: 0.4164
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6014 - accuracy: 0.7010
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3487 - accuracy: 0.6271
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.28

10/10 [==============================] - 0s 2ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 83/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0657 - accuracy: 0.9780
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0683 - accuracy: 0.9720
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0176 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1054 - accuracy: 0.9638
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0102 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 889us/step - loss: 0.0493 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0510 - accuracy: 0.9892
Epoch 90/200
10/10 [==============================] - 0s 889us/step - loss: 0.0339 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 92/

10/10 [==============================] - 0s 2ms/step - loss: 0.0403 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 889us/step - loss: 0.0214 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0391 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0508 - accuracy: 0.9918
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0506 - accuracy: 0.9642
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 889us/step - loss: 0.0600 - accuracy: 0.9780
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 1.0000
